# Chapter 18: Q&A, Summarization, and LLMs

In [65]:
!pip install transformers sentence-transformers rouge-score

In [ ]:
!wget https://raw.githubusercontent.com/dvgodoy/assets/main/PyTorchInPractice/parse_10K.py

In [2]:
from parse_10K import parse_10k_filing

## 18.3 EDGAR Dataset

![](https://raw.githubusercontent.com/dvgodoy/assets/main/PyTorchInPractice/images/ch0/data_step1.png)

In [ ]:
!wget https://github.com/dvgodoy/assets/releases/download/dataset/edgar10k_sp500_top25.tar.gz
!tar -xvzf edgar10k_sp500_top25.tar.gz
!mv filings edgar10k_sp500_top25

In [5]:
with open('./edgar10k_sp500_top25/sec-edgar-filings/MSFT/10-K/0001564590-22-026876/filing-details.html', 'r',
          encoding='utf-8') as f:
    html = f.read()

res = parse_10k_filing(html, 2)[0]
paragraphs = list(map(lambda s: s.strip(), filter(lambda s: len(s) > 10, res.split('  '))))

## 18.4 Hallucinations

In [25]:
hallucinated_answer = """
    As an S&P 500 company, uncertainies such as market fluctuations, changes in customer demand,
    and new competitors entering the market can be challenging to handle. Additionally, companies
    must also be prepared to adapt to changing trends and technological innovations, while ensuring
    that their products and services remain competitive and high-quality. Furthermore, companies
    must carefully evaluate their target market to remain relevant and generate revenue.
"""

![](https://raw.githubusercontent.com/dvgodoy/assets/main/PyTorchInPractice/images/ch0/data_step3.png)

In [15]:
import torch
from sentence_transformers import SentenceTransformer

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2', device=device)
embeddings = model.encode(paragraphs)
content = torch.as_tensor(embeddings)

In [ ]:
import torch.nn as nn

similarity = nn.CosineSimilarity(dim=1)

q = torch.as_tensor(model.encode([hallucinated_answer]))
similarities = similarity(q, content)
most = torch.topk(similarities, 5)

In [26]:
hallucination_context = '\n'.join([paragraphs[i] for i in sorted(most.indices)])
hallucination_context

'Competition in the technology sector Our competitors range in size from diversified global companies with significant research and development resources to small, specialized firms whose narrower product lines may let them be more effective in deploying technical, marketing, and financial resources. Barriers to entry in many of our businesses are low and many of the areas in which we compete evolve rapidly with changing and disruptive technologies, shifting user needs, and frequent introductions of new products and services. Our ability to remain competitive depends on our success in making innovative products, devices, and services that appeal to businesses and consumers.\nIt is uncertain whether our strategies will attract the users or generate the revenue required to succeed. If we are not effective in executing organizational and technical changes to increase efficiency and accelerate innovation, or if we fail to generate sufficient usage of our new products and services, we may n

![](https://raw.githubusercontent.com/dvgodoy/assets/main/PyTorchInPractice/images/ch0/model_step5.png)

In [27]:
from transformers import pipeline

summarizer = pipeline("summarization", model='sshleifer/distilbart-cnn-12-6', device=device)
hallucination_summary = summarizer(hallucination_context, min_length=100, max_length=250)
hallucination_summary

[{'summary_text': ' It is uncertain whether our strategies will attract the users or generate the revenue required to succeed . Barriers to entry in many of our businesses are low and many of the areas in which we compete evolve rapidly with changing and disruptive technologies, shifting user needs, and frequent introductions of new products and services . We make significant investments in products and . services that may not achieve expected returns . We also invest in the development and acquisition of a variety of hardware for productivity, communication, and entertainment including PCs, tablets, gaming devices, and . HoloLens .'}]

## 18.5 Asymmetric Semantic Search

In [28]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

asymmetric_model = SentenceTransformer('sentence-transformers/msmarco-MiniLM-L-12-v3', device=device)

In [29]:
asymmetric_embeddings = asymmetric_model.encode(paragraphs)

In [45]:
from sentence_transformers.util import semantic_search

query = "what are the sources of uncertainties?"
asymmetric_query = asymmetric_model.encode(query)

similarities = semantic_search(asymmetric_query, asymmetric_embeddings, top_k=5)[0]
similarities

[{'corpus_id': 36, 'score': 0.4358140230178833},
 {'corpus_id': 26, 'score': 0.3663944900035858},
 {'corpus_id': 0, 'score': 0.36041459441185},
 {'corpus_id': 31, 'score': 0.318837434053421},
 {'corpus_id': 25, 'score': 0.3184739053249359}]

In [46]:
most = [s['corpus_id'] for s in similarities]
most

[36, 26, 0, 31, 25]

![](https://raw.githubusercontent.com/dvgodoy/assets/main/PyTorchInPractice/images/ch0/data_step3.png)

In [47]:
asymmetric_context = '\n'.join([paragraphs[i] for i in sorted(most)])
asymmetric_context

'ITEM 1A. RISK FACTORS Our operations and financial results are subject to various risks and uncertainties, including those described below, that could adversely affect our business, financial condition, results of operations, cash flows, and the trading price of our common stock. STRATEGIC AND COMPETITIVE RISKS We face intense competition across all markets for our products and services, which may lead to lower revenue or operating margins.\nCYBERSECURITY, DATA PRIVACY, AND PLATFORM ABUSE RISKS Cyberattacks and security vulnerabilities could lead to reduced revenue, increased costs, liability claims, or harm to our reputation or competitive position.\nSecurity of our information technology Threats to IT security can take a variety of forms. Individual and groups of hackers and sophisticated organizations, including state-sponsored organizations or nation-states, continuously undertake attacks that pose threats to our customers and our IT. These actors may use a wide variety of methods

In [48]:
device = summarizer.device
summarizer.model.to('cpu')
summarizer.device = torch.device('cpu')

asymmetric_summary = summarizer(asymmetric_context, min_length=100, max_length=250)
asymmetric_summary

IndexError: ignored

In [49]:
max_len = summarizer.model.config.max_position_embeddings
max_len

1024

In [50]:
token_list = summarizer.tokenizer.encode(asymmetric_context, add_special_tokens=False)
len(token_list)

1589

In [51]:
len(asymmetric_context.split())/len(token_list)

0.8477029578351164

### 18.5.1 Trimmed Context

![](https://raw.githubusercontent.com/dvgodoy/assets/main/PyTorchInPractice/images/ch0/data_step3.png)

In [52]:
trimmed_context = summarizer.tokenizer.decode(token_list[:(max_len-2)])
trimmed_context

'ITEM 1A. RISK FACTORS Our operations and financial results are subject to various risks and uncertainties, including those described below, that could adversely affect our business, financial condition, results of operations, cash flows, and the trading price of our common stock. STRATEGIC AND COMPETITIVE RISKS We face intense competition across all markets for our products and services, which may lead to lower revenue or operating margins.\nCYBERSECURITY, DATA PRIVACY, AND PLATFORM ABUSE RISKS Cyberattacks and security vulnerabilities could lead to reduced revenue, increased costs, liability claims, or harm to our reputation or competitive position.\nSecurity of our information technology Threats to IT security can take a variety of forms. Individual and groups of hackers and sophisticated organizations, including state-sponsored organizations or nation-states, continuously undertake attacks that pose threats to our customers and our IT. These actors may use a wide variety of methods

![](https://raw.githubusercontent.com/dvgodoy/assets/main/PyTorchInPractice/images/ch0/model_step5.png)

In [54]:
summarizer.model.to('cuda')
summarizer.device = torch.device('cuda:0')

trimmed_summary = summarizer(trimmed_context, min_length=100, max_length=250)
trimmed_summary

[{'summary_text': ' Our operations and financial results are subject to various risks and uncertainties, including those described below . Cyberattacks and security vulnerabilities could lead to reduced revenue, increased costs, liability claims, or harm to our reputation or competitive position . We face intense competition across all markets for our products and services, which may lead to lower revenue or operating margins . The cyberattacks uncovered in late 2020 known as Solorigate or Nobelium are an example of a supply chain attack where malware was introduced to a software providers customers, including us, through software updates .'}]

### 18.5.2 Unsorted Context

![](https://raw.githubusercontent.com/dvgodoy/assets/main/PyTorchInPractice/images/ch0/data_step3.png)

In [55]:
unsorted_context = '\n'.join([paragraphs[i] for i in most])
unsorted_context

'Other digital safety abuses Our hosted consumer services as well as our enterprise services may be used to disseminate harmful or illegal content in violation of our terms or applicable law. We may not proactively discover such content due to scale, the limitations of existing technologies, and conflicting legal frameworks. When discovered by users, such content may negatively affect our reputation, our brands, and user engagement. Regulations and other initiatives to make platforms responsible for preventing or eliminating harmful content online have been enacted, and we expect this to continue. We may be subject to enhanced regulatory oversight, civil or criminal liability, or reputational damage if we fail to comply with content moderation regulations, adversely affecting our business and consolidated financial statements. The development of the IoT presents security, privacy, and execution risks. To support the growth of the intelligent cloud and the intelligent edge, we are devel

![](https://raw.githubusercontent.com/dvgodoy/assets/main/PyTorchInPractice/images/ch0/model_step5.png)

In [56]:
token_list = summarizer.tokenizer.encode(unsorted_context, add_special_tokens=False)
trimmed_unsorted_context = summarizer.tokenizer.decode(token_list[:(max_len-2)])
trimmed_unsorted_summary = summarizer(trimmed_unsorted_context, min_length=100, max_length=250)
trimmed_unsorted_summary

[{'summary_text': ' Our hosted consumer services as well as our enterprise services may be used to disseminate harmful or illegal content in violation of our terms or applicable law . We may not proactively discover such content due to scale, the limitations of existing technologies, and conflicting legal frameworks . The development of the IoT presents security, privacy, and execution risks . We are developing products, services, and technologies to power the IoT, a network of distributed and interconnected devices employing sensors, data, and computing capabilities including AI .'}]

## 18.6 ROUGE Score

![](https://raw.githubusercontent.com/dvgodoy/assets/main/PyTorchInPractice/images/ch0/model_step5.png)

![](https://raw.githubusercontent.com/dvgodoy/assets/main/PyTorchInPractice/images/ch16/ngrams.png)

In [57]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

In [58]:
scorer.score('nice to meet you here', 'nice to meet you now')

{'rouge1': Score(precision=0.8, recall=0.8, fmeasure=0.8000000000000002),
 'rouge2': Score(precision=0.75, recall=0.75, fmeasure=0.75),
 'rougeL': Score(precision=0.8, recall=0.8, fmeasure=0.8000000000000002)}

In [60]:
scores = scorer.score(hallucination_context,
                      hallucination_summary[0]['summary_text'].strip())
scores

{'rouge1': Score(precision=1.0, recall=0.12062256809338522, fmeasure=0.2152777777777778),
 'rouge2': Score(precision=0.9891304347826086, recall=0.11818181818181818, fmeasure=0.2111368909512761),
 'rougeL': Score(precision=0.8387096774193549, recall=0.10116731517509728, fmeasure=0.18055555555555555)}

In [61]:
scores = scorer.score(trimmed_context,
                      trimmed_summary[0]['summary_text'].strip())
scores

{'rouge1': Score(precision=1.0, recall=0.10411899313501144, fmeasure=0.18860103626943003),
 'rouge2': Score(precision=0.9666666666666667, recall=0.09965635738831616, fmeasure=0.18068535825545173),
 'rougeL': Score(precision=0.8571428571428571, recall=0.08924485125858124, fmeasure=0.1616580310880829)}

In [62]:
scores = scorer.score(trimmed_unsorted_context,
                      trimmed_unsorted_summary[0]['summary_text'].strip())
scores

{'rouge1': Score(precision=1.0, recall=0.09410430839002268, fmeasure=0.17202072538860105),
 'rouge2': Score(precision=0.975609756097561, recall=0.09080590238365494, fmeasure=0.16614745586708202),
 'rougeL': Score(precision=1.0, recall=0.09410430839002268, fmeasure=0.17202072538860105)}

## 18.7 Decoder-Based Models

![](https://raw.githubusercontent.com/dvgodoy/assets/main/PyTorchInPractice/images/ch16/blank1.png)

![](https://raw.githubusercontent.com/dvgodoy/assets/main/PyTorchInPractice/images/ch16/blank2.png)

![](https://raw.githubusercontent.com/dvgodoy/assets/main/PyTorchInPractice/images/ch16/fill2.png)

### 18.7.1 Generative Pretrained Transformer (GPT)

![](https://raw.githubusercontent.com/dvgodoy/assets/main/PyTorchInPractice/images/ch0/model_step5.png)

In [66]:
import torch
from transformers import pipeline

device = 0 if torch.cuda.is_available() else -1

generator = pipeline('text-generation', model = 'gpt2', device=device)

#### 18.7.1.1 Greedy Search

In [67]:
prompt = "what uncertainties are likely faced by an S&P500 company?"

generator(prompt, max_length=100, do_sample=False)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'what uncertainties are likely faced by an S&P500 company?\n\nThe S&P 500 is a very volatile market. It is not a safe place to be. It is not a safe place to be for investors. It is not a safe place to be for investors for the long term.\n\nThe S&P 500 is a very volatile market. It is not a safe place to be for investors. It is not a safe place to be for investors for the long term.'}]

#### 18.7.1.2 Beam Search

In [68]:
prompt = "what uncertainties are likely faced by an S&P500 company?"

generator(prompt, max_length=100, num_return_sequences=3, return_full_text=False, do_sample=True)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': " And are the major suppliers of food, fuel, drugs and other goods involved in the manufacturing of the food?\n\nThis column does not necessarily reflect the opinion of Bloomberg View's editorial board or Bloomberg LP, its owners and investors.\n\nTo contact the author of this story:\n\nEleanor F. Beecher on wbeecher@bloomberg.net\n\nTo contact the editor responsible for this story"},
 {'generated_text': ' What will happen in the future and can investors act quickly to get capital funding in order to help out as they see fit and be able to make long-term investment decisions for you [i.e., to buy your first stock], should you get ahead of the curve with your investment?\n\nG: Our goal with this program is to provide the investors with a foundation for understanding our market research process, which is based on the'},
 {'generated_text': '\n\nThis question came up on the "Ask Me Anything" panel of CNBC\'s Morning Money:\n\nWhat is not being worked on by an S&P500 co

#### 18.7.1.3 Temperature

In [69]:
prompt = "what uncertainties are likely faced by an S&P500 company?"

generator(prompt, max_length=100, num_return_sequences=3, return_full_text=False, do_sample=True, temperature=30.0)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': ' Would $12 trillion really equate very narrowly priced investments up ahead given "great global diversation of money managers," or an increased wealth share as measured across equity stakes but excluding leverAGE funds that pay in return more with high growth ratios while controlling losses relative an excess wealth split between unheditable businesses like General Motors as cash investors would. This can then result only somewhat misleading of capital, when at present this implies substantial excess assets for many'},
 {'generated_text': ' Why and through its long business as established UIT partner FMCSA\n… In order not even risk raising my name (i dont), is being included this way allows the person listed or someone to have such a low name… a number has no need with Firms having long trading name/investment/businesses from start(?) and start company through start till finish by default at various end stage for which these SPs get'},
 {'generated_text': ' Here at

In [70]:
prompt = "what uncertainties are likely faced by an S&P500 company?"

generator(prompt, max_length=100, num_return_sequences=3, return_full_text=False, do_sample=True, temperature=100.0)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': ' A financial uncertainty with possible significant future exposure should allow S$-value or equity trading volume adjustments but still should at worst present 10^13s. Moreover any loss margin over 9 years may need at this moment 0 0 which reduces returns/utility trading margin to its maximums 3\nFor trading income such at all or over this is required from 0 5 or the company capital base in relation that loss from it but before trading any'},
 {'generated_text': ' Could all major US investors be in jeopardy?", one investment forum contributor warned to CacuityInservienta CEO Craig Greenstein "there can definitely occur large negative short swings which may disrupt market activities". A couple additional questions would probably be "Would UCO and CFI [counterintelligence division offices across several major industry banks]- one analyst has proposed CSC/CI support from \'American intelligence firms as new means\' of counteroffensive'},
 {'generated_text': ' Share st

In [71]:
prompt = "what uncertainties are likely faced by an S&P500 company?"

generator(prompt, max_length=100, num_return_sequences=3, return_full_text=False, do_sample=True, temperature=1.0)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': " Is this a bad thing? No, because that's the only way to answer that question.\n\nThe other issues are the fact that there are very few clear indicators of the effect of the shift from fixed-price to price-to-value hedging, and other types — like the volatility in the performance of the equities market over the years after the financial crisis — would suggest to a person that the market is still quite"},
 {'generated_text': "\n\nNo, you are not alone. What's even more important to you is to think of some of the other risks of companies like Tesla. The fact is, the potential for a Tesla's stock price decline is very high, and the fact that the U.S. is experiencing a near-term stock market crash and no major price disruptions from a company like Tesla is an even greater hazard. The market should adjust this to"},
 {'generated_text': ' Here are a few reasons to take a look:\n\n(1) Over the past few years, the cost of S&P 500 stocks has been reduced in most regions of 

### 18.7.2 Inference API

![](https://raw.githubusercontent.com/dvgodoy/assets/main/PyTorchInPractice/images/ch0/model_step5.png)

#### 18.7.2.1 POST Request

In [72]:
import json
import requests
import os

os.environ['HFAPI_TOKEN'] = 'hf_...'

API_TOKEN = os.environ['HFAPI_TOKEN']  # assuming your API token is set

headers = {"Authorization": f"Bearer {API_TOKEN}"}
headers.update({'Content-Type': 'application/json'})

INFERENCE_ENDPOINT = 'https://api-inference.huggingface.co'

def query_inference_api(prompt, model='gpt2', **kwargs):
    url = f"{INFERENCE_ENDPOINT}/models/{model}"
    payload = {'inputs': prompt, 'parameters': {**kwargs}}
    data = json.dumps(payload)
    response = requests.request("POST", url, headers=headers, data=data)
    return json.loads(response.content.decode("utf-8"))

In [73]:
prompt = "what uncertainties are likely faced by an S&P500 company?"
generated_text = query_inference_api(prompt,
                                     max_length=100,
                                     num_return_sequences=3,
                                     return_full_text=False,
                                     do_sample=True,
                                     temperature=0.8)
generated_text

[{'generated_text': '\n\nNo:\n\nS&P500 companies are not subject to risk.\n\nWhat are the major risk factors for a S&P500 company?\n\nS&P500 companies are not subject to risk.\n\nWhat are the major risks associated with investing in a S&P500 company?\n\nS&P500 companies are not subject to risk.\n\nWhat are the key risks associated with investing'},
 {'generated_text': '\n\nLet me tell you something about our shareholders. In the past we were known as the three most important investors in our business. We have been a leading shareholder in the largest number of companies in the world — in fact, we have been a top shareholder at some of the biggest companies in the world — in fact, we hold the position of biggest shareholder in four of the top five largest corporations in the world. We have seen'},
 {'generated_text': ' And what do investors fear in a new market when companies are so likely to be bought by large shareholders, let alone by large shareholders?'}]

#### 18.7.2.2 Inference Client

In [74]:
from huggingface_hub import InferenceClient
import json

client = InferenceClient(token=API_TOKEN)

prompt = "what uncertainties are likely faced by an S&P500 company?"
params = {'max_length': 100,
          'num_return_sequences': 3,
          'return_full_text': False,
          'do_sample': True,
          'temperature': 0.8}

response = client.post(json={"inputs": prompt, "parameters": params}, model="gpt2")
json.loads(response)

[{'generated_text': '\n\nNo:\n\nS&P500 companies are not subject to risk.\n\nWhat are the major risk factors for a S&P500 company?\n\nS&P500 companies are not subject to risk.\n\nWhat are the major risks associated with investing in a S&P500 company?\n\nS&P500 companies are not subject to risk.\n\nWhat are the key risks associated with investing'},
 {'generated_text': '\n\nLet me tell you something about our shareholders. In the past we were known as the three most important investors in our business. We have been a leading shareholder in the largest number of companies in the world — in fact, we have been a top shareholder at some of the biggest companies in the world — in fact, we hold the position of biggest shareholder in four of the top five largest corporations in the world. We have seen'},
 {'generated_text': ' And what do investors fear in a new market when companies are so likely to be bought by large shareholders, let alone by large shareholders?'}]

## 18.8 Large Language Models (LLMs)

![](https://raw.githubusercontent.com/dvgodoy/assets/main/PyTorchInPractice/images/ch0/model_step5.png)

### 18.8.1 Instruction Models

#### 18.8.1.1 Falcon 7B Instruct

In [75]:
INFERENCE_ENDPOINT = 'https://api-inference.huggingface.co'

repo_id = "tiiuae/falcon-7b-instruct"

url = f"{INFERENCE_ENDPOINT}/models/{repo_id}"
url

'https://api-inference.huggingface.co/pipeline/text-generation/tiiuae/falcon-7b-instruct'

In [77]:
import os

os.environ['HFAPI_TOKEN'] = 'hf_...'

API_TOKEN = os.environ['HFAPI_TOKEN']  # assuming your API token is set

client = InferenceClient(url, token=API_TOKEN)

In [78]:
question = "what uncertainties are likely faced by an S&P500 company?"

prompt = f"""
You are an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.

{question}
"""

params = {"temperature": 0.8, "max_new_tokens": 500}

In [79]:
response = client.post(json={"inputs": prompt, "parameters": params})
json.loads(response)

[{'generated_text': "\nYou are an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.\n\nwhat uncertainties are likely faced by an S&P500 company?\nUncertainties that an S&P500 company is likely to face include:\n\n- Changes in market trends and customer preferences\n- Increased competition from new companies entering the market\n- Rising costs of production and technology\n- Legal and regulatory risks\n- Talent acquisition and retention challenges\n- Cybersecurity threats and data privacy issues\n\nIn order to stay competitive and relevant in the market, S&P500 companies must anticipate and adapt to these uncertainties. This may involve research and development, implementing new strategies, and keeping up with the changing market dynamics."}]

In [80]:
answer = json.loads(response)[0]['generated_text'].split('\n')[4:]
answer

['Uncertainties that an S&P500 company is likely to face include:',
 '',
 '- Changes in market trends and customer preferences',
 '- Increased competition from new companies entering the market',
 '- Rising costs of production and technology',
 '- Legal and regulatory risks',
 '- Talent acquisition and retention challenges',
 '- Cybersecurity threats and data privacy issues',
 '',
 'In order to stay competitive and relevant in the market, S&P500 companies must anticipate and adapt to these uncertainties. This may involve research and development, implementing new strategies, and keeping up with the changing market dynamics.']

### 18.8.2 Real Answers

#### 18.8.2.1 The Prompt
   
![](https://raw.githubusercontent.com/dvgodoy/assets/main/PyTorchInPractice/images/ch16/query_db.png)
   
![](https://raw.githubusercontent.com/dvgodoy/assets/main/PyTorchInPractice/images/ch16/llm_answer.png)

In [87]:
question = "what are the sources of uncertainties?"

context = """ITEM 1A. RISK FACTORS Our operations and financial results are subject to various risks and uncertainties, including those described below, that could adversely affect our business, financial condition, results of operations, cash flows, and the trading price of our common stock. STRATEGIC AND COMPETITIVE RISKS We face intense competition across all markets for our products and services, which may lead to lower revenue or operating margins.
It is uncertain whether our strategies will attract the users or generate the revenue required to succeed. If we are not effective in executing organizational and technical changes to increase efficiency and accelerate innovation, or if we fail to generate sufficient usage of our new products and services, we may not grow revenue in line with the infrastructure and development investments described above. This may negatively impact gross margins and operating income.
Public scrutiny of our decisions regarding user privacy, data practices, or content.
Challenging economic conditions also may impair the ability of our customers to pay for products and services they have purchased. As a result, allowances for doubtful accounts and write-offs of accounts receivable may increase.
Measures to contain a global pandemic may intensify other risks described in these Risk Factors. Any of these measures may adversely impact our ability to:
"""

prompt = f"""Answer the question as truthfully as possible using the provided text, and if the answer is not contained within the text below, say "I don't know".

Context:
{context}

{question}""".strip()

In [88]:
print(prompt)

Answer the question as truthfully as possible using the provided text, and if the answer is not contained within the text below, say "I don't know".

Context:
ITEM 1A. RISK FACTORS Our operations and financial results are subject to various risks and uncertainties, including those described below, that could adversely affect our business, financial condition, results of operations, cash flows, and the trading price of our common stock. STRATEGIC AND COMPETITIVE RISKS We face intense competition across all markets for our products and services, which may lead to lower revenue or operating margins.
It is uncertain whether our strategies will attract the users or generate the revenue required to succeed. If we are not effective in executing organizational and technical changes to increase efficiency and accelerate innovation, or if we fail to generate sufficient usage of our new products and services, we may not grow revenue in line with the infrastructure and development investments desc

#### 18.8.2.2 Falcon

In [89]:
params = {'temperature': 0.01, 'max_new_tokens': 500, 'return_full_text': False}

response = client.post(json={"inputs": prompt, "parameters": params})

In [90]:
answer = json.loads(response)[0]['generated_text']
print(answer)


The sources of uncertainties are the global pandemic, the competitive landscape, and the company's ability to execute its strategies and maintain its infrastructure and development investments.
